<a href="https://colab.research.google.com/github/CryptoRobotFr/General-code-trading-bot/blob/main/All-indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-binance

In [ ]:
pip install ta

In [ ]:
pip install pandas_ta

<h1>Import Dataset</h1>

In [32]:
import pandas as pd
from binance.client import Client
import ta
import pandas_ta as pda
import matplotlib.pyplot as plt
import numpy as np

client = Client()

klinesT = client.get_historical_klines("ETHUSDT", Client.KLINE_INTERVAL_1HOUR, "01 january 2021")

df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
df['close'] = pd.to_numeric(df['close'])
df['high'] = pd.to_numeric(df['high'])
df['low'] = pd.to_numeric(df['low'])
df['open'] = pd.to_numeric(df['open'])

df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']
print(df)

# ----- Functions Definition -------
def get_chop(high, low, close, window):
    tr1 = pd.DataFrame(high - low).rename(columns = {0:'tr1'})
    tr2 = pd.DataFrame(abs(high - close.shift(1))).rename(columns = {0:'tr2'})
    tr3 = pd.DataFrame(abs(low - close.shift(1))).rename(columns = {0:'tr3'})
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').dropna().max(axis = 1)
    atr = tr.rolling(1).mean()
    highh = high.rolling(window).max()
    lowl = low.rolling(window).min()
    ci = 100 * np.log10((atr.rolling(window).sum()) / (highh - lowl)) / np.log10(window)
    return ci

                        open     high  ...        tb_quote_av  ignore
timestamp                              ...                           
2021-01-01 00:00:00   736.42   739.00  ...  11015918.97221470       0
2021-01-01 01:00:00   734.08   749.00  ...  20362373.93350290       0
2021-01-01 02:00:00   748.27   749.00  ...  13207123.35208410       0
2021-01-01 03:00:00   744.06   747.23  ...   7079926.31087480       0
2021-01-01 04:00:00   744.87   747.09  ...   7539500.28687210       0
...                      ...      ...  ...                ...     ...
2021-08-20 07:00:00  3229.19  3233.00  ...  33516990.03284880       0
2021-08-20 08:00:00  3216.59  3227.31  ...  40078059.04603450       0
2021-08-20 09:00:00  3200.06  3217.90  ...  26269534.03612830       0
2021-08-20 10:00:00  3209.02  3225.00  ...  23890952.15151570       0
2021-08-20 11:00:00  3218.46  3218.67  ...  16242094.17269440       0

[5545 rows x 11 columns]


<h1>Backtest Execution</h1>

In [34]:
df.drop(df.columns.difference(['open','high','low','close','volume']), 1, inplace=True)

#Simple Moving Average
df['SMA']=ta.trend.sma_indicator(df['close'], window=12)

#Exponential Moving Average
df['EMA']=ta.trend.ema_indicator(close=df['close'], window=12)

#Relative Strength Index (RSI)
df['RSI'] =ta.momentum.rsi(close=df['close'], window=14)

#MACD
MACD = ta.trend.MACD(close=df['close'], window_fast=12, window_slow=26, window_sign=9)
df['MACD'] = MACD.macd()
df['MACD_SIGNAL'] = MACD.macd_signal()
df['MACD_DIFF'] = MACD.macd_diff() #Histogramme MACD

#Stochastic RSI
df['STOCH_RSI'] = ta.momentum.stochrsi(close=df['close'], window=14, smooth1=3, smooth2=3) #Non moyenné 
df['STOCH_RSI_D'] = ta.momentum.stochrsi_d(close=df['close'], window=14, smooth1=3, smooth2=3) #Orange sur TradingView
df['STOCH_RSI_K'] =ta.momentum.stochrsi_k(close=df['close'], window=14, smooth1=3, smooth2=3) #Bleu sur TradingView

#Ichimoku
df['KIJUN'] = ta.trend.ichimoku_base_line(high=df['high'], low=df['low'], window1=9, window2=26)
df['TENKAN'] = ta.trend.ichimoku_conversion_line(high=df['high'], low=df['low'], window1=9, window2=26)
df['SSA'] = ta.trend.ichimoku_a(high=df['high'], low=df['low'], window1=9, window2=26)
df['SSB'] = ta.trend.ichimoku_b(high=df['high'], low=df['low'], window2=26, window3=52)

#Bollinger Bands
BOL_BAND = ta.volatility.BollingerBands(close=df['close'], window=20, window_dev=2)
df['BOL_H_BAND'] = BOL_BAND.bollinger_hband() #Bande Supérieur
df['BOL_L_BAND'] = BOL_BAND.bollinger_lband() #Bande inférieur
df['BOL_MAVG_BAND'] = BOL_BAND.bollinger_mavg() #Bande moyenne

#Average True Range (ATR)
df['ATR'] = ta.volatility.average_true_range(high=df['high'], low=df['low'], close=df['close'], window=14)

#Super Trend
ST_length = 10
ST_multiplier = 3.0
superTrend = pda.supertrend(high=df['high'], low=df['low'], close=df['close'], length=ST_length, multiplier=ST_multiplier)
df['SUPER_TREND'] = superTrend['SUPERT_'+str(ST_length)+"_"+str(ST_multiplier)] #Valeur de la super trend
df['SUPER_TREND_DIRECTION'] = superTrend['SUPERTd_'+str(ST_length)+"_"+str(ST_multiplier)] #Retourne 1 si vert et -1 si rouge

#Awesome Oscillator
df['AWESOME_OSCILLATOR'] = ta.momentum.awesome_oscillator(high=df['high'], low=df['low'], window1=5, window2=34)

# Kaufman’s Adaptive Moving Average (KAMA)
df['KAMA'] = ta.momentum.kama(close=df['close'], window=10, pow1=2, pow2=30)

#Choppiness index
df['CHOP'] = get_chop(high=df['high'], low=df['low'], close=df['close'], window=14)  

df

,open,high,low,close,volume,SMA,EMA,RSI,MACD,MACD_SIGNAL,MACD_DIFF,STOCH_RSI,STOCH_RSI_D,STOCH_RSI_K,KIJUN,TENKAN,SSA,SSB,BOL_H_BAND,BOL_L_BAND,BOL_MAVG_BAND,ATR,SUPER_TREND,SUPER_TREND_DIRECTION,AWESOME_OSCILLATOR,KAMA,CHOP
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,736.42,739.00,729.33,734.07,27932.69884000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,734.165,NaN,NaN,NaN,0.000000,0.000000,1,NaN,NaN,NaN
2021-01-01 01:00:00,734.08,749.00,733.37,748.28,52336.18779000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,739.165,NaN,NaN,NaN,0.000000,NaN,1,NaN,NaN,NaN
2021-01-01 02:00:00,748.27,749.00,742.27,744.06,33019.50100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,739.165,NaN,NaN,NaN,0.000000,NaN,1,NaN,NaN,NaN
2021-01-01 03:00:00,744.06,747.23,743.10,744.82,17604.80859000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,739.165,NaN,NaN,NaN,0.000000,NaN,1,NaN,NaN,NaN
2021-01-01 04:00:00,744.87,747.09,739.30,742.29,18794.15424000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,739.165,NaN,NaN,NaN,0.000000,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-20 07:00:00,3229.19,3233.00,3206.61,3216.59,20882.29187000,3207.299167,3195.709055,68.120000,49.785174,44.653412,5.131762,0.220449,0.730705,0.610873,3103.485,3199.165,3151.3250,3100.965,3309.572169,2989.800831,3149.6865,33.112736,3129.634659,1,134.719118,3218.720439,33.094765
2021-08-20 08:00:00,3216.59,3227.31,3188.06,3200.07,25059.73381000,3210.796667,3196.379970,62.957796,46.445265,45.011782,1.433483,0.000000,0.584996,0.344281,3103.485,3210.380,3156.9325,3100.965,3304.959151,3014.594849,3159.7770,33.551112,3129.634659,1,126.439912,3217.935259,49.757181
2021-08-20 09:00:00,3200.06,3217.90,3195.18,3209.02,16563.44261000,3214.594167,3198.324590,64.526227,44.013197,44.812065,-0.798869,0.115695,0.355734,0.112048,3103.485,3215.280,3159.3825,3100.965,3300.929383,3037.424617,3169.1770,32.777462,3129.634659,1,118.586647,3217.622590,52.690884
